numpy=1.15.4
keras = 2.2.4
tensorFlow = 1.12.0 

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from keras.utils import np_utils
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

x_train = x_train.astype('float32')/255.
x_test = x_test.astype('float32')/255.

x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))



N, M = x_train.shape
O = 1024
P = 256
T = 10
batch_size = 100
learning_rate = 0.001
training_epochs = 10

inp = tf.placeholder(shape=(None, M), dtype=tf.float32)
out = tf.placeholder(shape=(None, T), dtype=tf.float32)

weights1 = tf.Variable(tf.truncated_normal([M, O], stddev=np.sqrt(1/M)))
bias1 = tf.Variable(tf.ones([O])/O)
layer1 = tf.nn.relu(tf.matmul(inp, weights1) + bias1)

weights2 = tf.Variable(tf.truncated_normal([O, P], stddev=np.sqrt(1/O)))
bias2 = tf.Variable(tf.ones([P])/P)
layer2 = tf.nn.relu(tf.matmul(layer1, weights2) + bias2)

weights_output = tf.Variable(tf.truncated_normal([P, T], stddev=np.sqrt(1/P)))
bias_output = tf.Variable(tf.ones([T])/T)
evidence = tf.matmul(layer2, weights_output) + bias_output
output = tf.nn.softmax(evidence)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=evidence, labels= out)
cross_entropy = tf.reduce_mean(cross_entropy)

train_step = tf.train.RMSPropOptimizer(learning_rate).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(out, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar("Cost", cross_entropy)
tf.summary.scalar("Accuracy", accuracy)
summary_op = tf.summary.merge_all()

with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(training_epochs):
            print("Epoch: ", epoch)
            batch_count = int(N/batch_size)
            for i in range(batch_count):
                start, finish = batch_size*i, batch_size*(i+1)
                batch_x, batch_y = x_train[start:finish, :], y_train[start:finish, :]
                d = {inp: batch_x, out: batch_y}
                _, summary = sess.run([train_step, summary_op], feed_dict=d)
        print("Accuracy", accuracy.eval(feed_dict={inp: x_test, out: y_test}))

Using TensorFlow backend.


Epoch:  0
Epoch:  1


KeyboardInterrupt: 

In [2]:
import numpy as np
import tensorflow as tf

from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense

(x_train, y_train), (x_test, y_test) = mnist.load_data()

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

x_train = x_train.astype('float32')/255.
x_test = x_test.astype('float32')/255.

x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))

N, M = x_train.shape
O = 1024
P = 256
T = 10
batch_size = 100
learning_rate = 0.001
training_epochs = 10

model = Sequential()
model.add(Dense(P, activation='relu'))
model.add(Dense(T, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=training_epochs, batch_size=batch_size)
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print(score)

Epoch 1/10
60000/60000 [==============================] - 13s 209us/step - loss: 0.2730 - acc: 0.9213
Epoch 2/10
60000/60000 [==============================] - 11s 191us/step - loss: 0.1204 - acc: 0.9650
Epoch 3/10
60000/60000 [==============================] - 13s 209us/step - loss: 0.0822 - acc: 0.9762
Epoch 4/10
60000/60000 [==============================] - 13s 223us/step - loss: 0.0624 - acc: 0.9811
Epoch 5/10
60000/60000 [==============================] - 15s 250us/step - loss: 0.0500 - acc: 0.9854
Epoch 6/10
60000/60000 [==============================] - 16s 270us/step - loss: 0.0402 - acc: 0.9883
Epoch 7/10
60000/60000 [==============================] - 17s 291us/step - loss: 0.0329 - acc: 0.9907
Epoch 8/10
60000/60000 [==============================] - 18s 301us/step - loss: 0.0271 - acc: 0.9923
Epoch 9/10
60000/60000 [==============================] - 17s 284us/step - loss: 0.0217 - acc: 0.9934
Epoch 10/10
10000/10000 [==============================] - 1s 126us/step
[0.070238